In [22]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.model_selection import StratifiedKFold
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop


# data input 
path = 'trainrat_new.txt'
data = pd.read_csv(path,sep=' ')
print(len(data))

# Data extraction,Win
win_data = data[data.label == "W"]
win_data = win_data.assign(W=1,D=0,L=0) 
win_data = win_data.drop("label",axis=1)
win_data = win_data.sample(n=len(win_data))# random sort

# Data extraction,Draw
draw_data = data[data.label == "D"]
draw_data = draw_data.assign(W=0,D=1,L=0)
draw_data = draw_data.drop("label",axis=1)
draw_data = draw_data.sample(n=len(draw_data))# random sort

# Data extraction,Lose
lose_data = data[data.label == "L"]
lose_data = lose_data.assign(W=0,D=0,L=1) 
lose_data = lose_data.drop("label",axis=1)
lose_data = lose_data.sample(n=len(lose_data))# random sort

# Group data together,use for normal k-fold
all_data = pd.concat([win_data, draw_data, lose_data])
all_data = all_data.sample(n=len(all_data))# random sort
#yAll = xAll[["W","D","L"]]

# Data drop
#xWin = xWin.drop(["W","D","L"],axis=1)
#xDraw = xDraw.drop(["W","D","L"],axis=1)
#xLose = xLose.drop(["W","D","L"],axis=1)
#xAll = xAll.drop(["W","D","L"],axis=1)

# Data separate and making dataset
win_data_separate = []
draw_data_separate = []
lose_data_separate = []
all_data_separate = []
wdl_separate = []
for i in range(10):
    win_data_separate.append(win_data[i::10])
    draw_data_separate.append(draw_data[i::10])
    lose_data_separate.append(lose_data[i::10])
    all_data_separate.append(all_data[i::10])
    # merge for stratified sampling
    wdl_separate.append(pd.concat([win_data_separate[i],draw_data_separate[i],lose_data_separate[i]]))
    # assign a number to make final input data
    wdl_separate[i] = wdl_separate[i].assign(separate_num=i)
    all_data_separate[i] = all_data_separate[i].assign(separate_num=i)

# integrate everything once
wdl_separate_merge = wdl_separate[0]
all_data_separate_merge = all_data_separate[0]
for i in range(1,10):
    wdl_separate_merge = wdl_separate_merge.append(wdl_separate[i])
    all_data_separate_merge = all_data_separate_merge.append(all_data_separate[i])

#print(len(wdl_separate_merge))
#print(len(all_data_separate_merge))
    
    
# make final input data
x_train = []
y_train = []
x_test = []
y_test = []
xAll_train = []
yAll_train = []
xAll_test = []
yAll_test = []
for i in range(10):
    x_train.append(wdl_separate_merge[wdl_separate_merge['separate_num'] != i])
    x_test.append(wdl_separate_merge[wdl_separate_merge['separate_num'] == i])
    xAll_train.append(all_data_separate_merge[all_data_separate_merge['separate_num'] != i])
    xAll_test.append(all_data_separate_merge[all_data_separate_merge['separate_num'] == i])
for i in range(10):
    # delete separate_num
    x_train[i] = x_train[i].drop(['separate_num'],axis=1)
    x_test[i] = x_test[i].drop(['separate_num'],axis=1)
    xAll_train[i] = xAll_train[i].drop(['separate_num'],axis=1)
    xAll_test[i] = xAll_test[i].drop(['separate_num'],axis=1)
    # random sort
    x_train[i] = x_train[i].sample(n=len(x_train[i]))
    x_test[i] = x_test[i].sample(n=len(x_test[i]))
    xAll_train[i] = xAll_train[i].sample(n=len(xAll_train[i]))
    xAll_test[i] = xAll_test[i].sample(n=len(xAll_test[i]))
    
    # separate x and y
    y_train.append(x_train[i][["W","D","L"]])
    y_test.append(x_test[i][["W","D","L"]])
    yAll_train.append(xAll_train[i][["W","D","L"]])
    yAll_test.append(xAll_test[i][["W","D","L"]])
    x_train[i] = x_train[i].drop(["W","D","L"],axis=1)
    x_test[i] = x_test[i].drop(["W","D","L"],axis=1)
    xAll_train[i] = xAll_train[i].drop(["W","D","L"],axis=1)
    xAll_test[i] = xAll_test[i].drop(["W","D","L"],axis=1)


print(x_train[0].head())
print(y_train[0].head())
print(xAll_train[0].head())
print(yAll_test[0].head())


#print(len(y_train[0].query('W == 1')))
#print(len(y_train[0].query('L == 1')))
#print(len(y_train[0].query('D == 1')))
#print(len(yAll_train[0].query('W == 1')))
#print(len(yAll_train[0].query('L == 1')))
#print(len(yAll_train[0].query('D == 1')))




28186
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
8786   0.923317  0.881482  0.170981  0.943447  2.672943 -0.710455  1.865760   
7852  -0.755908  0.496644 -0.145410  0.585298  0.444773 -0.125090  1.022363   
2910   0.343985  0.017492 -0.034297  0.373169  0.298621  0.115688 -0.355079   
9918   5.664396 -0.118996  1.640708 -0.576552  0.813753  0.008538  0.017193   
14322  0.546451 -0.414521  0.604462 -0.338395 -0.021979 -0.187426  0.192316   

          AADEF  
8786  -0.760125  
7852  -0.014479  
2910   0.768625  
9918   0.996157  
14322 -0.133027  
       W  D  L
8786   0  0  1
7852   0  0  1
2910   1  0  0
9918   1  0  0
14322  1  0  0
          HHATT     HHDEF     HAATT     HADEF     AHATT     AHDEF     AAATT  \
4356   0.092412 -0.041416 -0.034599 -0.006496  0.162067 -0.051721 -0.037287   
26581  0.183477  0.304217 -0.214807  0.489547 -0.122499  0.369468  0.033757   
7550  -0.032407  0.726275  0.367550  0.897207  0.107485  0.251516 -0.309932   
21212

In [18]:
# create model
modelArelu = Sequential()
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(10, activation='relu'))
modelArelu.add(Dense(1, activation='sigmoid'))

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true), axis=-1)

def RPS(y_true, y_pred):
    siz = tf.size(y_true)
    cumulative_sum_y_true = 0.
    cumulative_sum_y_pred = 0.
    output = 0.
    for i in siz-1:
        cumulative_sum_y_true += y_true[i]
        cumulative_sum_y_pred += y_pred[i]
        output += (cumulative_sum_y_true - cumulative_sum_y_pred) ** 2
        
    output /= (siz-1)
    return output
        

modelArelu.compile(loss='binary_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [26]:
cvscores = []
batch_size = 16
num_classes = 10
epochs = 10
for train, test in kfold.split(x, y):
    print("TRAIN:", train, "TEST:", test)
    X_train, X_test = x.iloc[train], x.iloc[test]
    y_train, y_test = y.iloc[train], y.iloc[test]

    # Fit the model
    modelArelu.fit(X_train.values, y_train.values,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1
                  )


    results = modelArelu.predict(X_test, batch_size=batch_size, verbose=0, steps=None)
    
    print(results)
    print(y_test.values)


TRAIN: [    0     1     3 ... 28183 28184 28185] TEST: [    2     8    17 ... 28166 28173 28176]
Epoch 1/10
25367/25367 [==============================] - 1s 37us/step - loss: 0.6569 - acc: 0.6059
Epoch 2/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6515 - acc: 0.6147
Epoch 3/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6509 - acc: 0.6170
Epoch 4/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6506 - acc: 0.6153
Epoch 5/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6503 - acc: 0.6142
Epoch 6/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6499 - acc: 0.6164
Epoch 7/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6500 - acc: 0.6158
Epoch 8/10
25367/25367 [==============================] - 1s 31us/step - loss: 0.6495 - acc: 0.6173
Epoch 9/10
25367/25367 [==============================] - 1s 30us/step - loss: 0.6498 - acc: 0.6157
Epo

In [1]:
'''Trains a simple deep NN on the MNIST dataset.
Gets to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import StratifiedKFold
import numpy

fold_num = 10
batch_size = 128
num_classes = 10
epochs = 20

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step
60000 train samples
10000 test samples
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)          